In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn import cluster
import os
import re
# import seaborn as sns

import warnings
warnings.filterwarnings('ignore')
os.getcwd()


'c:\\Users\\dscshap3808\\Documents\\my_scripts_new\\play1'

In [30]:
df = pd.read_csv('./daily_ou_kpi.csv')
# df.dropna(axis = 1, inplace = True, how = 'all')
# re1 = re.compile(r'(?<=\.).+')
# df.columns = [re1.findall(i)[0] for i in list(df.columns.to_numpy())]
# df.to_csv('./dws_dsc_wh_ou_daily_kpi_july.csv', encoding='utf_8_sig')

# df.to_csv('./daily_kpi_all_810.csv', encoding='utf_8_sig')

In [31]:
re1 = re.compile(r'(?<=\.).+')
df.columns = [re1.findall(i)[0] for i in list(df.columns.to_numpy())]

In [35]:
df[df['ou_code'] == 'CN-070'].head()

,ou_code,operation_day,inbound_receive_qty,is_holiday,outbound_shipped_qty,total_head_count,total_working_hour,outsource_working_hour,perm_working_hour,other_working_hour,direct_working_hour,indirect_working_hour,outbound_inbound_qty_ratio,perm_working_hour_ratio,working_hour_per_head,location_usage_rate,location_idle_rate
3913,CN-070,20210414,0.0,0,29851.0,75.0,820.96,0.00,0.00,0.0,807.59,13.37,0.0,0.000000,10.946133,0.000000,0.000000
3914,CN-070,20210702,0.0,0,133415.0,85.0,864.42,474.11,390.31,0.0,847.37,17.05,0.0,0.451528,10.169647,0.067992,0.932008
3915,CN-070,20210407,0.0,0,253417.0,78.0,738.33,0.00,0.00,0.0,722.22,16.11,0.0,0.000000,9.465769,0.000000,0.000000
3916,CN-070,20210622,0.0,0,192648.0,87.0,985.70,503.66,482.04,0.0,969.43,16.27,0.0,0.489033,11.329885,0.099423,0.900577
3917,CN-070,20210411,0.0,1,0.0,59.0,715.99,0.00,0.00,0.0,715.99,0.00,0.0,0.000000,12.135424,0.000000,0.000000


In [29]:

# clean_df2 = df.groupby('ou_code')[[
#     'inbound_receive_qty', 'outbound_shipped_qty'
#     ]].sum().reset_index()
# clean_df2['sum'] = clean_df2.sum(axis = 1)
# clean_df2 = clean_df2[clean_df2['sum'] != 0]
# df = df[df['ou_code'].isin(clean_df2.ou_code)]

In [34]:

clean_df1 = (df.groupby('ou_code')['operation_day'].count() <2).reset_index()
clean_df1.columns = ['ou_code', 'flag1']
df = clean_df1.merge(df, on = 'ou_code', how = 'inner')
df = df[df['flag1'] == False]

clean_df2 = df.groupby('ou_code')[[
    'inbound_receive_qty', 'outbound_shipped_qty'
    ]].sum().reset_index()
clean_df2['sum'] = clean_df2.sum(axis = 1)
clean_df2 = clean_df2[clean_df2['sum'] != 0]
df = df[df['ou_code'].isin(clean_df2.ou_code)]

clean_df3 = (df.groupby('ou_code')[[
    'total_working_hour'
    ]].sum() == 0).reset_index()
clean_df3 = clean_df3[clean_df3['total_working_hour'] == False]


# clean_df4 = (df.groupby('ou_code')[[
#     'outsource_working_hour'
#     ]].sum() == 0).reset_index()
# clean_df4 = clean_df4[clean_df4['outsource_working_hour'] == False]
# df = df[df['ou_code'].isin(clean_df4.ou_code)]
# df= df.reset_index()
 
# df = df[df['ou_code'].isin(clean_df3.ou_code)]
# df= df.reset_index()

df = df[[
    'ou_code','operation_day', 'inbound_receive_qty', 'is_holiday',
    'outbound_shipped_qty','total_head_count','total_working_hour',
    'outsource_working_hour', 'perm_working_hour',
    'other_working_hour', 'direct_working_hour', 'indirect_working_hour',
    'outbound_inbound_qty_ratio', 'perm_working_hour_ratio',
    'working_hour_per_head', 'location_usage_rate', 'location_idle_rate']]
df = df.fillna(0)
df = df[df['total_working_hour'] != 0]
df.head()

,ou_code,operation_day,inbound_receive_qty,is_holiday,outbound_shipped_qty,total_head_count,total_working_hour,outsource_working_hour,perm_working_hour,other_working_hour,direct_working_hour,indirect_working_hour,outbound_inbound_qty_ratio,perm_working_hour_ratio,working_hour_per_head,location_usage_rate,location_idle_rate
0,CN-001,20210430,0.0,0,1699.0,4.0,32.75,0.0,0.00,0.0,32.75,0.0,0.000000,0.0,8.1875,0.000000,0.000000
1,CN-001,20210619,0.0,1,0.0,3.0,30.00,0.0,30.00,0.0,30.00,0.0,0.000000,1.0,10.0000,0.011628,0.988372
2,CN-001,20210427,4794.0,0,7392.0,4.0,32.28,0.0,0.00,0.0,32.28,0.0,1.541927,0.0,8.0700,0.000000,0.000000
3,CN-001,20210419,11400.0,0,6000.0,4.0,33.25,0.0,0.00,0.0,33.25,0.0,0.526316,0.0,8.3125,0.000000,0.000000
4,CN-001,20210617,114.0,0,13527.0,5.0,40.58,0.0,40.58,0.0,40.58,0.0,118.657895,1.0,8.1160,0.001600,0.998400


In [5]:
len(df['ou_code'].unique())

65

In [6]:
from sklearn.metrics import davies_bouldin_score
from sklearn.cluster import KMeans 

In [7]:
# from sklearn.metrics import davies_bouldin_scores

def mnb_kmeans_in(ou_code):
        """
        mini batch kmeans, inbound, outbound, working hour data.
        simple algorithm, adding cols {max, min, mean, median, 75 quantile, distance to kernal}
        Calculate Davies Bouldin score
        from sklearn.metrics import davies_bouldin_score
        null data fill
        """

        df_fin = pd.DataFrame()
        df_sub = df[df['ou_code'] == ou_code][['ou_code', 'operation_day', 'inbound_receive_qty']]        
        df_fin = df_fin.append(df_sub[df_sub['inbound_receive_qty'] == 0])
        df_fin['kernal_core1' ] = -1
        df_fin['kernal_value1'] = 0

        """
        not null data training
        """
        df_rec = df_sub[df_sub['inbound_receive_qty'] != 0]      
        scores = []
        for center in list(range(2,30)):
            kmeans = cluster.KMeans(n_clusters=center)
            model  = kmeans.fit_predict(
                        X = np.reshape(list(df_rec['inbound_receive_qty']), (-1,1))
                        )
            score  = davies_bouldin_score(
                        np.reshape(list(df_rec['inbound_receive_qty']), (-1,1)), model
                        )
            scores.append(score)

        center = np.argmin(scores)
        print(center)
        alg1 = cluster.MiniBatchKMeans(n_clusters = center, random_state = 529)
        hist1 = alg1.fit(np.reshape(list(df_rec['inbound_receive_qty']), (-1,1)))
        df_rec['kernal_core1'] = hist1.labels_
        cl_1 = pd.concat(
                [pd.DataFrame(hist1.cluster_centers_), pd.Series(np.arange(0,center))], axis = 1
                )
        
        cl_1.columns = ['kernal_value1', 'kernal_core1']
        df_rec = df_rec.merge(
                cl_1, on = 'kernal_core1', how = 'inner'
                )
        
        """
        merging
        """
        df_fin = df_fin.append(df_rec).reset_index().drop(['index'], axis = 1)

        # df_fin['kind'] = 'inbound'

        return df_fin



# scores = []
# centers = list(range(2,30))
# for center in centers:
#     scores.append(get_kmeans_score(cluster_df, center))
 

In [ ]:
df_fin = pd.DataFrame()
df_sub = df[df['ou_code'] == 'CN-298'][['ou_code', 'operation_day', 'inbound_receive_qty']]        
df_fin = df_fin.append(df_sub[df_sub['inbound_receive_qty'] == 0])
df_fin['kernal_core1' ] = -1
df_fin['kernal_value1'] = 0
"""
not null data training
"""
df_rec = df_sub[df_sub['inbound_receive_qty'] != 0]      
scores = []
for center in list(range(2,10)):
    kmeans = cluster.KMeans(n_clusters=center)
    model  = kmeans.fit_predict(X = np.reshape(list(df_rec['inbound_receive_qty']), (-1,1)))
    score  = davies_bouldin_score(np.reshape(list(df_rec['inbound_receive_qty']), (-1,1)), model)
    scores.append(score)
center = min(scores)
np.argmin(scores)

In [ ]:
mnb_kmeans_in('CN-214')

In [8]:

def mnb_kmeans_in(ou_code):
        """
        mini batch kmeans, inbound, outbound, working hour data.
        simple algorithm, adding cols {max, min, mean, median, 75 quantile, distance to kernal}
        """
        alg1 = cluster.MiniBatchKMeans(n_clusters = 5, random_state = 5290403)
        """
        null data fill
        """
        df_fin = pd.DataFrame()
        df_sub = df[df['ou_code'] == ou_code][['ou_code', 'operation_day', 'inbound_receive_qty']]        
        df_fin = df_fin.append(df_sub[df_sub['inbound_receive_qty'] == 0])
        df_fin['kernal_core1' ] = -1
        df_fin['kernal_value1'] = 0
        """
        not null data training
        """
        df_rec = df_sub[df_sub['inbound_receive_qty'] != 0]      
        hist1 = alg1.fit(np.reshape(list(df_rec['inbound_receive_qty']), (-1,1)))
        df_rec['kernal_core1'] = hist1.labels_
        cl_1 = pd.concat(
                [pd.DataFrame(hist1.cluster_centers_), pd.Series(np.arange(0,5))], axis = 1
                )
        
        cl_1.columns = ['kernal_value1', 'kernal_core1']
        df_rec = df_rec.merge(
                cl_1, on = 'kernal_core1', how = 'inner'
                )
        
        """
        merging
        """
        df_fin = df_fin.append(df_rec).reset_index().drop(['index'], axis = 1)

        # df_fin['kind'] = 'inbound'

        return df_fin


def mnb_kmeans_out(ou_code):
        alg1 = cluster.MiniBatchKMeans(n_clusters = 5, random_state = 5290403)
        df_fin = pd.DataFrame()
        df_sub = df[df['ou_code'] == ou_code][['ou_code', 'operation_day', 'outbound_shipped_qty']]        
        df_fin = df_fin.append(df_sub[df_sub['outbound_shipped_qty'] == 0])
        df_fin['kernal_core2' ] = -1
        df_fin['kernal_value2'] = 0
        df_rec = df_sub[df_sub['outbound_shipped_qty'] != 0]

        hist1 = alg1.fit(np.reshape(list(df_rec['outbound_shipped_qty']), (-1,1)))

        df_rec['kernal_core2'] = hist1.labels_
        cl_1 = pd.concat(
                [pd.DataFrame(hist1.cluster_centers_), pd.Series(np.arange(0,5))], axis = 1
                )
        
        cl_1.columns = ['kernal_value2', 'kernal_core2']

        df_rec = df_rec.merge(
                cl_1, on = 'kernal_core2', how = 'inner'
                )
        df_fin = df_fin.append(df_rec).reset_index().drop(['index'], axis = 1)
        # df_fin['kind'] = 'outbound'
        return df_fin

def mnb_kmeans_hr(ou_code):
        alg1 = cluster.MiniBatchKMeans(n_clusters = 5, random_state = 5290403)
        df_fin = pd.DataFrame()
        df_sub = df[df['ou_code'] == ou_code][['ou_code', 'operation_day', 'total_working_hour']]        
        df_fin = df_fin.append(df_sub[df_sub['total_working_hour'] == 0])
        df_fin['kernal_core3' ] = -1
        df_fin['kernal_value3'] = 0
        df_rec = df_sub[df_sub['total_working_hour'] != 0]

        hist1 = alg1.fit(np.reshape(list(df_rec['total_working_hour']), (-1,1)))

        df_rec['kernal_core3'] = hist1.labels_
        cl_1 = pd.concat(
                [pd.DataFrame(hist1.cluster_centers_), pd.Series(np.arange(0,5))], axis = 1
                )
        
        cl_1.columns = ['kernal_value3', 'kernal_core3']

        df_rec = df_rec.merge(
                cl_1, on = 'kernal_core3', how = 'inner'
                )
        df_fin = df_fin.append(df_rec).reset_index().drop(['index'], axis = 1)
        """
        组内kernal distance 
        """
        df_fin['dis_core']=df_fin.groupby(
                'kernal_core3'
                )['total_working_hour','kernal_value3'].diff(
                        axis = 1
                        ).drop('total_working_hour', axis = 1).round(3)
        
        return df_fin



In [9]:
len(df['ou_code'].unique())


65

In [10]:
ou_codes = list(df['ou_code'].unique())
p = list()
for i in ou_codes:
    try: 
        mnb_kmeans_in(i)
        mnb_kmeans_out(i)
        # mnb_kmeans_hr(i)
        # mnb_kmeans_hr2(i)
    except:
        
        p.append(i)


for i in p:
    ou_codes.remove(i)
[p, len(p), len(ou_codes)]

[['CN-070', 'CN-392', 'HK-015'], 3, 62]

In [20]:
%%time
from functools import reduce
df_final = pd.DataFrame()
for i in ou_codes:
        df_final = df_final.append(
            reduce(
                lambda left,right: pd.merge(
                    left,right,on= ['ou_code', 'operation_day']
                ), [mnb_kmeans_in(i), mnb_kmeans_out(i), mnb_kmeans_hr(i)]
                )
    )

"""
for loop , 对所有ou进行独立的kmeans on inb qty and outb qty
随后merge 原始表
"""

# np.setdiff1d(ou_codes,df_final.ou_code.unique())
df_final = df_final.merge(
    df[['ou_code','operation_day',\
        'outbound_inbound_qty_ratio','working_hour_per_head','total_head_count','is_holiday']],
    on = ['ou_code', 'operation_day'],
    how = 'left'
    )


Wall time: 13 s


In [12]:
# df_final.groupby(
#                 'kernal_core3'
#                 )['total_working_hour','kernal_value3'].diff(axis = 1).drop('total_working_hour', axis = 1)

In [15]:
# df_final['outsource_working_hour']

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
8144    0.0
8145    0.0
8146    0.0
8147    0.0
8148    0.0
Name: outsource_working_hour, Length: 8149, dtype: float64

In [21]:
def mnb_kmeans_hr2(ou_code):
        alg1 = cluster.MiniBatchKMeans(n_clusters = 5, random_state = 5290403)
        df_fin = pd.DataFrame()
        df_sub = df[df['ou_code'] == ou_code][['ou_code', 'operation_day', 'outsource_working_hour']]  
        df_fin = df_fin.append(df_sub[df_sub['outsource_working_hour'] == 0])
        df_fin['kernal_core4' ] = -1
        df_fin['kernal_value4'] = 0
        df_rec = df_sub[df_sub['outsource_working_hour'] != 0]
 
        hist1 = alg1.fit(np.reshape(list(df_rec['outsource_working_hour']), (-1,1)))

        df_rec['kernal_core4'] = hist1.labels_
        cl_1 = pd.concat(
                [pd.DataFrame(hist1.cluster_centers_), pd.Series(np.arange(0,5))], axis = 1
                )
        
        cl_1.columns = ['kernal_value4', 'kernal_core4']

        df_rec = df_rec.merge(
                cl_1, on = 'kernal_core4', how = 'outer'
                )
        df_fin = df_fin.append(df_rec).reset_index().drop(['index'], axis = 1)
        
        df_fin['dis_core_os']=df_fin.groupby(
                'kernal_core4'
                )['outsource_working_hour','kernal_value4'].diff(
                        axis = 1
                        ).drop('outsource_working_hour', axis = 1).round(3)
        return df_fin


ou_codes
p = list()
for i in ou_codes:
    try: 
        # mnb_kmeans_in(i)
        # mnb_kmeans_out(i)
        # mnb_kmeans_hr(i)
        mnb_kmeans_hr2(i)
    except:
        
        p.append(i)


for i in p:
    ou_codes.remove(i)
 

In [22]:
[len(p),len(ou_codes)]

[0, 33]

In [23]:
df_final2 = pd.DataFrame()
for i in ou_codes:
        df_final2 = df_final2.append(
            mnb_kmeans_hr2(i)
        )

df_final = df_final.merge(df_final2, on =  ['ou_code', 'operation_day'], how = 'left').fillna(0)

In [24]:
df_final[df_final['dis_core_os']!=0]

,ou_code,operation_day,inbound_receive_qty,kernal_core1,kernal_value1,outbound_shipped_qty,kernal_core2,kernal_value2,total_working_hour,kernal_core3,kernal_value3,dis_core,outbound_inbound_qty_ratio,working_hour_per_head,total_head_count,is_holiday,outsource_working_hour,kernal_core4,kernal_value4,dis_core_os
6,CN-002,20210625,0.0,-1,0.000000e+00,12437.0,1,12440.139151,40.42,1,40.870852,0.451,0.000000,8.084000,5.0,0,16.30,0,16.369848,0.070
7,CN-002,20210708,0.0,-1,0.000000e+00,5893.0,3,6737.728291,57.73,2,55.509891,-2.220,0.000000,11.546000,5.0,0,21.70,2,21.703955,0.004
10,CN-002,20210826,0.0,-1,0.000000e+00,6866.0,3,6737.728291,32.70,4,33.208525,0.509,0.000000,8.175000,4.0,0,16.52,0,16.369848,-0.150
14,CN-002,20210903,0.0,-1,0.000000e+00,2955.0,0,2215.048520,40.55,1,40.870852,0.321,0.000000,8.110000,5.0,0,16.35,0,16.369848,0.020
15,CN-002,20210707,0.0,-1,0.000000e+00,2514.0,0,2215.048520,57.68,2,55.509891,-2.170,0.000000,11.536000,5.0,0,21.88,2,21.703955,-0.176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4677,CN-417,20210829,2881509.0,0,2.881509e+06,0.0,-1,0.000000,206.68,2,208.854467,2.174,0.000000,12.917500,16.0,1,195.50,0,186.724177,-8.776
4678,CN-417,20210903,106909.0,4,1.028484e+05,501.0,3,384.390558,88.41,1,56.422029,-31.988,0.004686,8.841000,10.0,0,63.00,1,70.209677,7.210
4679,CN-417,20210803,100000.0,4,1.028484e+05,0.0,-1,0.000000,202.25,2,208.854467,6.604,0.000000,9.630952,21.0,0,193.50,0,186.724177,-6.776
4680,CN-417,20210824,151671.0,3,1.426260e+05,153.0,0,155.783920,177.15,0,184.355871,7.206,0.001009,8.435714,21.0,0,167.50,4,163.938735,-3.561


In [ ]:
# model = sm.OLS(Y = X)
# results = model.fit()
# results.params

In [ ]:
import seaborn as sns

import plotly.graph_objects as go
import plotly.express as px

# wide_df = px.data.medals_wide()

# fig = px.bar(wide_df, x="nation", y=["gold", "silver", "bronze"], title="Wide-Form Input")

In [ ]:
plt

In [ ]:
from datetime import datetime
import time

plt = df_final[df_final['dis_core_os']!=0]
plt['operation_day'] = [datetime.strptime(i,'%Y%m%d').strftime('%Y%m%d') for i in plt['operation_day'].astype(str)]
plt['operation_day']= pd.to_datetime(plt['operation_day'])

In [ ]:
# # # sns.barplot(x = 'operation_day', y = ['inbound_receive_qty', 'outbound_shipped_qty'],  \
# # #     data=df_final.iloc[0:30,:], color = 'red'
# # #     ) 
# # # # g.set(xlim=(1, 20210805))  
# g = px.bar(plt[plt['ou_code'] == 'CN-298'],\
#     x = 'operation_day', y = ['inbound_receive_qty', 'outbound_shipped_qty']);g


In [ ]:
plt2 = plt[plt['ou_code'] == 'CN-298']
plt2['outsource_working_hour'] = plt2['outsource_working_hour'] * 100
plt2 = plt2.sort_values('operation_day')

In [ ]:
g = go.Figure([go.Bar(x = plt2['operation_day'] ,y = plt2['inbound_receive_qty']),\
    go.Bar(x = plt2['operation_day'], y = plt2['outbound_shipped_qty'])])
g.update_layout(barmode='stack')
g.add_trace(go.Scatter(x =plt2['operation_day'], y = plt2['outsource_working_hour'], mode ='lines'))
# g.add_trace(go.Scatter(plt[plt['ou_code'] == 'CN-298'], x = 'operation_day', y =  'outsource_working_hour'))

In [ ]:
def mutatation_(new_col, calculate_methdo):
    df_final[new_col] = df_final.groupby(
    ['ou_code', 'kernal_core1', 'kernal_core2']
    )['total_working_hour'].transform(calculate_methdo)
    return df_final


In [ ]:
plt2['dis_core'].describe()

In [ ]:
px.box(plt2, y = ['dis_core',  'dis_core_os'])

In [ ]:
go.Figure([go.Histogram(x = plt2['dis_core']),\
    go.Histogram(x= plt2['dis_core_os'])])
 

In [28]:

df_final.head()
df_final['max_wh'] = df_final.groupby(
    ['ou_code', 'kernal_core1', 'kernal_core2']
    )['total_working_hour'].transform('max')
df_final['min_wh'] = df_final.groupby(
    ['ou_code', 'kernal_core1', 'kernal_core2']
    )['total_working_hour'].transform('min')
df_final['median_wh'] = df_final.groupby(
    ['ou_code', 'kernal_core1', 'kernal_core2']
    )['total_working_hour'].transform('median')
df_final['mean_wh'] = df_final.groupby(
    ['ou_code', 'kernal_core1', 'kernal_core2']
    )['total_working_hour'].transform('mean')
df_final['qt_66_wh'] = df_final.groupby(
    ['ou_code', 'kernal_core1', 'kernal_core2']
    )['total_working_hour'].transform('quantile', .6667)
df_final['qt_75_wh'] = df_final.groupby(
    ['ou_code', 'kernal_core1', 'kernal_core2']
    )['total_working_hour'].transform('quantile', .75)
df_final['d_to_core_outer'] = df_final.groupby(
        ['ou_code', 'kernal_core1', 'kernal_core2'], as_index = False
                )['total_working_hour','kernal_value3'].diff(axis = 1
                    ).drop('total_working_hour', axis = 1).round(3)
df_final['d_to_core_outer_os'] = df_final.groupby(
        ['ou_code', 'kernal_core1', 'kernal_core2'], as_index = False
                )['outsource_working_hour','kernal_value4'].diff(axis = 1
                    ).drop('outsource_working_hour', axis = 1).round(3)
df_final['d_to_core_outer_os'] = np.abs(df_final['d_to_core_outer_os'])

df_final['percent_error_66'] = (
        df_final['qt_66_wh'] - df_final['total_working_hour'])/(
                df_final['total_working_hour']
                )
df_final['percent_error_75'] = (
        df_final['qt_75_wh'] - df_final['total_working_hour']
        )/(df_final['total_working_hour']
        )

df_final['qt_75_os'] = df_final.groupby(
    ['ou_code', 'kernal_core1', 'kernal_core2']
    )['outsource_working_hour'].transform('quantile', .75)
    

df_final['pe_66_os'] = (
        df_final['qt_66_wh'] - df_final['outsource_working_hour'])/(
                df_final['outsource_working_hour']
                )
df_final['pe_75_os'] = (
        df_final['qt_75_wh'] - df_final['outsource_working_hour']
        )/(df_final['outsource_working_hour']
        )


df_final['qt_75_dis_core_os_inner'] = df_final.groupby(
    ['ou_code', 'kernal_core4']
    )['dis_core_os'].transform('quantile', .75)

df_final['qt_75_dis_core_os_outer'] = df_final.groupby(
    ['ou_code', 'kernal_core1', 'kernal_core2']
    )['d_to_core_outer_os'].transform('quantile', .75)

df_final['flag_wh'] = [1 if df_final['dis_core_os'][i]>df_final['qt_75_dis_core_os_outer'][i]\
     else 0 for i in np.arange(0, len(df_final))]

# df_final['date_stamp'] = str(date.today())
# df_final['date_stamp'] = df_final['date_stamp'].str.replace('-', '')
 
# df_final['inc_day']  = '99991231'

In [26]:
df_final[df_final['ou_code'] == 'CN-214']['flag_wh']

2358    0
2359    0
2360    0
2361    1
2362    0
       ..
2504    0
2505    0
2506    0
2507    0
2508    0
Name: flag_wh, Length: 151, dtype: int64

In [27]:
df_final =df_final.replace(float('inf'), 0); df_final

,ou_code,operation_day,inbound_receive_qty,kernal_core1,kernal_value1,outbound_shipped_qty,kernal_core2,kernal_value2,total_working_hour,kernal_core3,...,d_to_core_outer,d_to_core_outer_os,percent_error_66,percent_error_75,qt_75_os,pe_66_os,pe_75_os,qt_75_dis_core_os_inner,qt_75_dis_core_os_outer,flag_wh
0,CN-002,20210604,0.0,-1,0.000000e+00,6004.0,3,6737.728291,41.04,1,...,-0.169,0.000,-0.001543,-0.000305,16.445,0.000000,0.000000,0.000,0.003,0
1,CN-002,20210515,0.0,-1,0.000000e+00,0.0,-1,0.000000,27.83,0,...,0.024,0.000,0.446283,0.450593,14.380,0.000000,0.000000,0.000,0.000,0
2,CN-002,20210407,0.0,-1,0.000000e+00,6537.0,3,6737.728291,40.53,1,...,0.341,0.000,0.011021,0.012275,16.445,0.000000,0.000000,0.000,0.003,0
3,CN-002,20210619,0.0,-1,0.000000e+00,0.0,-1,0.000000,13.33,3,...,0.000,0.000,2.019508,2.028507,14.380,8.044955,8.071910,0.000,0.000,0
4,CN-002,20210420,0.0,-1,0.000000e+00,510.0,0,2215.048520,40.85,1,...,0.021,0.000,0.001226,0.005630,16.360,0.000000,0.000000,0.000,0.030,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4677,CN-417,20210829,2881509.0,0,2.881509e+06,0.0,-1,0.000000,206.68,2,...,2.174,-8.776,0.000000,0.000000,195.500,0.057187,0.057187,7.349,-8.776,0
4678,CN-417,20210903,106909.0,4,1.028484e+05,501.0,3,384.390558,88.41,1,...,-31.988,7.210,0.000000,0.000000,63.000,0.403333,0.403333,12.460,7.210,0
4679,CN-417,20210803,100000.0,4,1.028484e+05,0.0,-1,0.000000,202.25,2,...,6.604,-6.776,0.000000,0.000000,193.500,0.045220,0.045220,7.349,-6.776,0
4680,CN-417,20210824,151671.0,3,1.426260e+05,153.0,0,155.783920,177.15,0,...,7.206,-3.561,0.020398,0.022947,168.250,0.079185,0.081881,2.939,-3.811,1


In [ ]:
# [1 if a > .25 else 0 for a in df_final[df_final['ou_code'] == 'CN-035']['percent_error_75']]


In [ ]:

# df_final['percent_error_66'] = (
#         df_final['qt_66_wh'] - df_final['total_working_hour'])/(
#                 df_final['total_working_hour']
#                 )
# df_final['percent_error_75'] = (
#         df_final['qt_75_wh'] - df_final['total_working_hour']
#         )/(df_final['total_working_hour']
#         )

# # df_final = df_final.assign(percent_error_66 = [a-1 if a == 1 else a for a in df_final['percent_error_66']])
# # df_final = df_final.assign(percent_error_75 = [a-1 if a == 1 else a for a in df_final['percent_error_75']])


In [ ]:
# df_final[df_final['percent_error_75'] != 0].dropna()
from datetime import date
df_final['inc_day'] = str(date.today())
df_final['inc_day'] = df_final['inc_day'].str.replace('-', '')

In [ ]:
# df_final.to_csv('./wh_in_out_fin.csv', index= None)

In [ ]:
df_final[df_final['percent_error_75'] < -.05]

---

---

----

In [ ]:
tes = df_final.merge(df_final.groupby('ou_code').agg({
    'inbound_receive_qty': ['std'],
    'outbound_shipped_qty': ['std'],
    'outsource_working_hour': ['std']
    }
).reset_index(), on = 'ou_code', how = 'left')


# .agg({'RECEIPT_ID':['nunique'], 
#         'TOTAL_LINES': ['sum'], 
#         'TOTAL_CONTAINERS': 'sum', 
#         'TOTAL_WEIGHT':'sum',
#         'TOTAL_QTY': ['sum'],
#         'CREATION_DATE_TIME_STAMP_2': 'min' }).reset_index()

In [ ]:
tes = df_final.groupby('ou_code').agg({
    'inbound_receive_qty': ['std'],
    'outbound_shipped_qty': ['std'],
    'outsource_working_hour': ['std']
    }
).reset_index()

In [ ]:
tes.columns = 

In [ ]:
%%time

str(date.today()).replace('-', '')

In [ ]:
df_final

In [ ]:
df_final['kernal_core1'].astype(str) + df_final['kernal_core2'].astype(str) 

In [ ]:
 import plotly.express as px
 def plot3d(ou_code):
    df_plot = df_final[df_final['ou_code'] == ou_code]
    df_plot['kernal_core5'] = df_final['kernal_core1'].astype(str) + df_final['kernal_core2'].astype(str) 
    fig = px.scatter_3d(df_plot, x='inbound_receive_qty', y='outbound_shipped_qty', z='total_working_hour',
              color = 'kernal_core3')
    return fig.show()


In [ ]:
plot3d('CN-214')

In [ ]:

fig.show()